In [20]:
from nltk import word_tokenize

In [21]:
# exp.1
sent1="I love sky, I love sea."
sent2="I like running, I love reading."
# step.1 分词
sents=[sent1,sent2]
texts=[[word for word in word_tokenize(sent)]for sent in sents]
print(texts)

[['I', 'love', 'sky', ',', 'I', 'love', 'sea', '.'], ['I', 'like', 'running', ',', 'I', 'love', 'reading', '.']]


In [22]:
# step.2 构建语料库
all_list=[]
for text in texts:
    all_list+=text
corpus=set(all_list)
print(corpus)

{'reading', 'like', ',', 'sky', 'running', 'sea', 'I', 'love', '.'}


In [23]:
# step.3 对语料库中的单词及标点建立数字映射，便于后续的句子的向量表示。
corpus_dict=dict(zip(corpus,range(len(corpus))))
print(corpus_dict)

{'reading': 0, 'like': 1, ',': 2, 'sky': 3, 'running': 4, 'sea': 5, 'I': 6, 'love': 7, '.': 8}


In [24]:
# step.4 建立句子的向量表示
# 这个表示向量并不是简单地以单词或标点出现与否来选择0，1数字，
# 而是把单词或标点的出现频数作为其对应的数字表示
def vector_rep(text, corpus_dict):
    vec=[]
    for key in corpus_dict.keys():
        if key in text:
            vec.append((corpus_dict[key],text.count(key)))
        else:
            vec.append((corpus_dict[key],0))
    vec=sorted(vec,key=lambda x:x[0])
    return vec

vec1=vector_rep(sents[0],corpus_dict)
vec2=vector_rep(sents[1],corpus_dict)
print(vec1)
print(vec2)

[(0, 0), (1, 0), (2, 1), (3, 1), (4, 0), (5, 1), (6, 2), (7, 2), (8, 1)]
[(0, 1), (1, 1), (2, 1), (3, 0), (4, 1), (5, 0), (6, 2), (7, 1), (8, 1)]


In [25]:
# 以上即为词袋模型
# 计算句子向量间的相似度
#     一般会选择余弦相似度
from math import sqrt
# 计算余弦相似度
def similarity_with_2_sents(vec1,vec2):
    inner_product=0
    square_vec1=0
    square_vec2=0
    for tup1,tup2 in zip(vec1,vec2):
        inner_product += tup1[1]*tup2[1]
        square_vec1+=tup1[1]**2
        square_vec2+=tup2[1]**2
    return (inner_product/sqrt(square_vec1*square_vec2))

cosin_sim=similarity_with_2_sents(vec1,vec2)
print(cosin_sim)

0.7302967433402214


In [27]:
"""
调包实现上述功能
"""
# 调用gensim模块
from gensim import corpora
from gensim.similarities import Similarity

# 语料库
dictionary = corpora.Dictionary(texts)

# 利用doc2bow作为词袋模型
corpus=[dictionary.doc2bow(text) for text in texts]
similarity = Similarity('-Similarity-index',corpus,num_features=len(dictionary))
print(similarity)

Similarity index with 2 documents in 0 shards (stored under -Similarity-index)


In [29]:
# 获取句子相似度
new_sensence = sent1
test_corpus_1 = dictionary.doc2bow(word_tokenize(new_sensence))
cosin_sim=similarity[test_corpus_1][1]
print("利用gensim计算得到两个句子的相似度： %.4f。"%cosin_sim)

利用gensim计算得到两个句子的相似度： 0.7303。
